In [39]:
import pandas as pd
import requests
import time

In [2]:
data = pd.read_excel("Bronchiectasis_Mastersheet_Postcode.xlsx")

In [3]:
data.head()

,UID,Gender,age,PostCode,Dead or Alive,Date of Death,NTM,NTM sp,PSE,Others,...,haemoptysis,Azithromycin,PPI,MRC score,significant comorbidities,Morphological Appearance,DISTRIBUTION OF THE DISEASE ON THE CT SCAN,SEVERITY ON CT SCAN,OTHER FEATURES ON THE SCAN,NOTES
0,#001,M,68,BA15 1NG,A,NaT,N,NONE,N,Y,...,N,N,N,1.0,Y,Not described,"Patchy/multi-lovar (>1 lobe, not all)",Mild,None of above mentioned,NaN
1,#002,M,76,BA3 3TQ,A,NaT,Y,MYCGOR,Y,Y,...,Y,N,Y,4.0,Y,Not described,Not described,Not described,None of above mentioned,NaN
2,#003,M,72,BA11 4BU,A,NaT,N,NaN,Y,Y,...,Y,Y,N,3.0,N,Not described,"Patchy/multi-lovar (>1 lobe, not all)",Mild,None of above mentioned,NaN
3,#004,F,82,BA4 5YG,D,2017-08-20 01:00:00,N,NaN,Y,Y,...,Y,Y,N,5.0,Y,Not described,"Patchy/multi-lovar (>1 lobe, not all)",Severe,None of above mentioned,NaN
4,#005,F,64,BA14 9TE,A,NaT,N,NaN,N,Y,...,Y,Y,N,1.0,N,Not described,Not described,Not described,Tree in bud,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 24 columns):
UID                                           312 non-null object
Gender                                        312 non-null object
age                                           312 non-null int64
PostCode                                      312 non-null object
Dead or Alive                                 312 non-null object
Date of Death                                 19 non-null datetime64[ns]
NTM                                           239 non-null object
NTM sp                                        236 non-null object
PSE                                           286 non-null object
Others                                        285 non-null object
First org                                     185 non-null object
Sputum purulence                              277 non-null object
Sputum quantity                               273 non-null object
cough                          

In [5]:
data = data[['UID','PostCode','NTM','PSE']]

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 4 columns):
UID         312 non-null object
PostCode    312 non-null object
NTM         239 non-null object
PSE         286 non-null object
dtypes: object(4)
memory usage: 4.9+ KB


In [7]:
data['NTM'].value_counts()

N    196
Y     43
Name: NTM, dtype: int64

In [8]:
data['PSE'].value_counts()

Y    180
N    106
Name: PSE, dtype: int64

In [9]:
ntm_dummies = pd.get_dummies(data['NTM'], prefix="NTM")

In [10]:
pse_dummies = pd.get_dummies(data['PSE'], prefix="PSE")

In [11]:
data.drop(['PSE', 'NTM'], inplace=True, axis=1)

In [12]:
data = pd.concat([data, pse_dummies, ntm_dummies], axis=1)

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 6 columns):
UID         312 non-null object
PostCode    312 non-null object
PSE_N       312 non-null uint8
PSE_Y       312 non-null uint8
NTM_N       312 non-null uint8
NTM_Y       312 non-null uint8
dtypes: object(2), uint8(4)
memory usage: 3.7+ KB


In [40]:
def get_long(postcode):
    try:
        postcode = postcode.replace(" ", "+")
        response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={}'.format(postcode))
        resp_json_payload = response.json()
        return resp_json_payload['results'][0]['geometry']['location']['lng']
    except:
        time.sleep(2)
        return get_long(postcode)

In [43]:
def get_lat(postcode):
    try:
        postcode = postcode.replace(" ", "+")
        response = requests.get('https://maps.googleapis.com/maps/api/geocode/json?address={}'.format(postcode))
        resp_json_payload = response.json()
        return resp_json_payload['results'][0]['geometry']['location']['lat']
    except:
        time.sleep(2)
        return get_long(postcode)

In [41]:
data['long'] = data.PostCode.apply(get_long)

In [44]:
data['lat'] = data.PostCode.apply(get_lat)

In [45]:
data

,UID,PostCode,PSE_N,PSE_Y,NTM_N,NTM_Y,lat,long
0,#001,BA15 1NG,1,0,1,0,-2.254453,-2.254453
1,#002,BA3 3TQ,0,1,0,1,51.288376,-2.456001
2,#003,BA11 4BU,0,1,1,0,51.220027,-2.354251
3,#004,BA4 5YG,0,1,1,0,51.193556,-2.553936
4,#005,BA14 9TE,1,0,1,0,-2.229979,-2.229979
5,#006,BA11 2HR,0,1,1,0,51.241612,-2.313030
6,#007,BS31 2AY,0,1,0,1,51.417362,-2.502424
7,#008,SN10 1SL,0,1,1,0,51.324131,-2.084894
8,#009,SN14 7QZ,0,1,1,0,51.493719,-2.218693
9,#010,SN12 6HA,0,1,1,0,-2.140341,-2.140341


In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 312 entries, 0 to 311
Data columns (total 8 columns):
UID         312 non-null object
PostCode    312 non-null object
PSE_N       312 non-null uint8
PSE_Y       312 non-null uint8
NTM_N       312 non-null uint8
NTM_Y       312 non-null uint8
lat         312 non-null float64
long        312 non-null float64
dtypes: float64(2), object(2), uint8(4)
memory usage: 8.6+ KB


In [47]:
data.to_csv("loc_data.csv", index=False)

In [48]:
import gmplot

In [52]:
pyo = data[data['PSE_Y'] == 1]
ntm = data[data['NTM_Y'] == 1]
all_other = data[(data['PSE_Y'] == 0) & (data['NTM_Y'] == 0)]

In [60]:
gmap = gmplot.GoogleMapPlotter(51.3758, -2.3599, 11)
gmap.scatter(all_other['lat'].tolist(), all_other['long'].tolist(), '#3B0B39', size=40, marker=True)
gmap.scatter(pyo['lat'].tolist(), pyo['long'].tolist(), '#32CD32', size=40, marker=False)
gmap.scatter(ntm['lat'].tolist(), ntm['long'].tolist(), '#ff0000', size=40, marker=False)
gmap.draw("all_samples.html")

In [57]:
gmap_hm = gmplot.GoogleMapPlotter(51.3758, -2.3599, 11)
gmap_hm.plot(ntm['lat'].tolist(), ntm['long'].tolist())
gmap_hm.draw("ntm_heatmap.html")